# Generate all figures

Generate all the figures included in the paper
*Relax and Recover: Guaranteed Range-Only Continuous Localization*.

In [ ]:
from itertools import cycle
import time

import matplotlib.pyplot as plt
import matplotlib
import numpy as np

import sys
sys.path.append('source/')

import hypothesis as h

%matplotlib inline
%reload_ext autoreload
%autoreload 2

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

from distutils.spawn import find_executable
if find_executable('latex'):
    print('using latex support')
    matplotlib.rcParams['text.usetex'] = True

In [ ]:
from plotting_tools import make_dirs_safe

directory = 'figures/'
make_dirs_safe(directory)
lines = ["-", "--", "-.", ":"]
linecycler = cycle(lines)

## Figure 4

In [ ]:
from trajectory_creator import get_trajectory
from plotting_tools import add_scalebar, remove_ticks
from plotting_tools import savefig

np.random.seed(0)

fig, axis = plt.subplots(1, 4, figsize=(5, 1.2), sharey='row',
                        gridspec_kw={'hspace': 0, 'wspace': 0.1})
traj = get_trajectory("clover")
traj.scale_bounding_box([5, 5])
error = np.random.normal(size=traj.coeffs.shape)
error = error/np.linalg.norm(error)
# todo scale the trajectory
for idx, scale in enumerate([0.1, 0.2, 0.4, 0.8]):
    ax = axis[idx]
    traj.plot(ax=ax, linestyle='--')
    perturbed = traj.copy()
    perturbed.coeffs += scale*error
    perturbed.plot(ax=ax)
    ax.axis('equal')
    ax.set_title("error: {}m".format(scale))
    add_scalebar(ax, loc='upper left', size=1)
    remove_ticks(ax)
    
savefig(fig, directory+"error_overview.pdf")

## Figure 5

In [ ]:
from plotting_tools import plot_noise
plt.rcParams['figure.figsize'] = 5, 2.5

fig, axis = plt.subplots(2, 1, figsize=(5, 4), sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

plot = plot_noise('noise_and_anchors',
                  error_types=['errors'],
                  save_figures=True,
                  min_noise=0,
                  max_noise=None,
                  background_alpha=0.2,
                  start=30,
                  anchors=True,
                  ax=axis[1])
plot = plot_noise('noise_right_inverse_weighted',
                  error_types=['errors'],
                  save_figures=False,
                  max_noise=None,
                  background_alpha=0.2,
                  start=100,
                  ax=axis[0])
plot.set_xticks([31, 62, 125, 250, 500])
axis[0].set_xlim(15, 500)
axis[1].set_ylim(1e-4, 1e-1)
axis[0].set_ylim(5e-4, 5e1)
axis[0].grid()
axis[1].grid()
axis[0].set_ylabel('errors')
axis[1].set_xlabel("number of measurements")
plt.tight_layout()

savefig(fig, directory + 'noise.pdf')

## Figure 3

In [ ]:
n_measurement_list = range(10, 40)

experiment_params={
    "n_dimensions": 2,
    "n_anchors": 4,
    "n_constraints": 5,
    "n_anchors_list": [3, 4, 5],
    "n_constraints_list": [3, 5, 7],
}

start = time.time()
probabilities_anchors = []
for idx, n_anchors in enumerate(experiment_params["n_anchors_list"]):
    print("{} anchors".format(n_anchors))
    probabilities_anchors.append([h.probability_upper_bound(
        n_dimensions=experiment_params["n_dimensions"],
        n_constraints=experiment_params["n_constraints"],
        n_measurements=n,
        position_wise=False,
        n_anchors=n_anchors,
        n_positions=np.Infinity,
        full_matrix=True
    ) for n in n_measurement_list])
probabilities_anchors = np.array(probabilities_anchors)
print("anchors total time: {:.2f}s".format(time.time()-start))

start = time.time()
probabilities_constraints = []
for idx, n_constraints in enumerate(experiment_params["n_constraints_list"]):
    print("{} constraints".format(n_constraints))
    probabilities_constraints.append([h.probability_upper_bound(
        n_dimensions=experiment_params["n_dimensions"],
        n_constraints=n_constraints,
        n_measurements=n,
        position_wise=False,
        n_anchors=experiment_params["n_anchors"],
        n_positions=np.Infinity,
        full_matrix=True,
    ) for n in n_measurement_list])
probabilities_constraints = np.array(probabilities_constraints)
print("constrains total time: {:.2f}s".format(time.time()-start))

In [ ]:
fig, axis = plt.subplots(2, 1, figsize=(5, 4), sharex='col', gridspec_kw={'hspace': 0.1, 'wspace': 0})

linecycler = cycle(lines)
for idx, n_anchors in enumerate(experiment_params["n_anchors_list"]):

    p = axis[0].step(
        n_measurement_list,
        probabilities_anchors[idx],
        label="{} anchors".format(n_anchors),
        where='post',
        linestyle=next(linecycler))
axis[0].set_ylabel("probability")
axis[0].grid()
axis[0].legend(loc='lower right')

linecycler = cycle(lines)
for idx, n_constraints in enumerate(experiment_params["n_constraints_list"]):

    p = axis[1].step(
        n_measurement_list,
        probabilities_constraints[idx],
        label="K={}".format(n_constraints),
        where='post',
        linestyle=next(linecycler))
axis[1].set_xlabel("number of measurements")
axis[1].set_ylabel("probability")
axis[1].grid()
axis[1].legend()
plt.tight_layout()

savefig(fig, directory+'probability.pdf')

## Figure 6

In [ ]:
from scipy.io import loadmat
from trajectory_creator import get_trajectory
from public_data_utils import read_dataset, get_plotting_params

plt.rcParams['figure.figsize'] = 7, 3

verbose = False
anchor_names = None  # use all anchors by default.
filename = 'datasets/Plaza2.mat' # triangle. works well.

full_df, anchors_df, traj = read_dataset(filename)
xlim, ylim = get_plotting_params(filename)

In [ ]:
from public_data_utils import get_ground_truth
from evaluate_dataset import compute_distance_matrix, compute_anchors
chosen_df = full_df
chosen_distance = 'distance'
range_system_id = 'Range'
#chosen_distance = 'distance_gt'

## Construct anchors. 
anchors = compute_anchors(anchors_df, anchor_names)

## Construct times.
times = chosen_df[chosen_df.system_id == range_system_id].timestamp.unique()

## Construct D.
D, times = compute_distance_matrix(chosen_df, anchors_df, anchor_names, times, chosen_distance)
if np.sum(D > 0) > D.shape[0]:
    print('Warning: multiple measurements for times:{}/{}!'.format(
          np.sum(np.sum(D > 0, axis=1)>1), D.shape[0]))

## Construct ground truth.
ground_truth_pos = get_ground_truth(chosen_df, times)

### Figure 6 first row 

In [ ]:
from other_algorithms import pointwise_srls
from plotting_backup import plot_complexities_old

list_complexities = [3, 5, 21, 51]
fig_size = [5, 1.2]
ylim = [-15, 75]

srls = False
rls = True

anchors = anchors[:2, :]
fig, axs = plot_complexities_old(traj, D, times, anchors, full_df, 
                             list_complexities, srls=srls, rls=rls)
[ax.set_xlim(*xlim) for ax in axs]
[ax.set_ylim(*ylim) for ax in axs]
fig.set_size_inches(*fig_size) 

if srls:
    savefig(fig, 'figures/results_with_srls.pdf')
elif rls:
    savefig(fig, 'figures/results_with_rls.pdf')
else:
    savefig(fig, 'figures/results.pdf')

### Figure 6 second row

In [ ]:
from plotting_backup import plot_subsample_old

fig_size = [5, 1.2]

n_complexity = 5
traj.set_n_complexity(n_complexity)

min_number = n_complexity*(traj.dim + 2) - 1 
#n_measurements_list = np.arange(D.shape[0], min_number, step=-100)
#n_measurements_list = [19, 20, 30, 40, 50, 100, 200, 300, 499]
n_measurements_list = [19, 30, 60, 200][::-1]

srls = False
rls = True

fig, axs = plot_subsample_old(traj, D, times, anchors, full_df, 
                          n_measurements_list, srls=srls, rls=rls)
[ax.set_xlim(*xlim) for ax in axs]
[ax.set_ylim(*ylim) for ax in axs]
fig.set_size_inches(*fig_size)

savefig(fig, 'figures/downsampling.pdf')